In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



In [6]:
#Preprocecing
#First of all we will check how many null values there are in each column
df = pd.read_csv('chess_games.csv')


In [9]:
df_first_lines = df.head(50000)
df_first_lines.to_csv('shorterChessDf.csv', index=False)

In [ ]:
# Display the data types of each column
print(df.dtypes)

In [ ]:
# Counting null values in each column
null_counts = df.isnull().sum()

print(null_counts)


In [ ]:

# Counting unique values in each column
unique_counts = df.nunique()

print(unique_counts)